# 공구 플랫폼 거래상태 및 운영 인사이트 분석


## 라이브러리 임포트


In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
import plotly.io as pio
import warnings

from datetime import datetime, timedelta
from plotly.subplots import make_subplots

In [2]:
warnings.filterwarnings('ignore')
pio.renderers.default = "vscode"

## 데이터 로드 및 기본 정보


In [3]:
# 데이터 로드
group_boards = pd.read_csv('../data/group_boards_dummy_data_500.csv')
participants = pd.read_csv('../data/participants_dummy_data_3000.csv')

print(f"Group Boards: {group_boards.shape[0]:,}행 × {group_boards.shape[1]}열")
print(f"Participants: {participants.shape[0]:,}행 × {participants.shape[1]}열")

# 기본 정보 출력
print("\nGroup Boards 컬럼 정보:")
print(group_boards.info())

print("\nParticipants 컬럼 정보:")
print(participants.info())

# 샘플 데이터 확인
print("\nGroup Boards 샘플 데이터:")
print(group_boards.head())

print("\nParticipants 샘플 데이터:")
print(participants.head())

Group Boards: 500행 × 10열
Participants: 3,000행 × 7열

Group Boards 컬럼 정보:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   group_board_id    500 non-null    int64 
 1   title             500 non-null    object
 2   content           500 non-null    object
 3   location          500 non-null    object
 4   status            500 non-null    object
 5   deadline          500 non-null    object
 6   created_at        500 non-null    object
 7   updated_at        500 non-null    object
 8   total_users       500 non-null    int64 
 9   group_product_id  500 non-null    int64 
dtypes: int64(3), object(7)
memory usage: 30.0+ KB
None

Participants 컬럼 정보:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 


## 데이터 전처리


In [4]:
# 날짜 컬럼 변환
group_boards['created_at'] = pd.to_datetime(group_boards['created_at'])
group_boards['deadline'] = pd.to_datetime(group_boards['deadline'])
group_boards['updated_at'] = pd.to_datetime(group_boards['updated_at'])
participants['joined_at'] = pd.to_datetime(participants['joined_at'])

# 월 컬럼 추가
group_boards['created_month'] = group_boards['created_at'].dt.to_period('M')
participants['joined_month'] = participants['joined_at'].dt.to_period('M')

# 공구 기간 계산
group_boards['duration_days'] = (group_boards['deadline'] - group_boards['created_at']).dt.days

# 요일 정보 추가
group_boards['created_weekday'] = group_boards['created_at'].dt.day_name()
participants['joined_weekday'] = participants['joined_at'].dt.day_name()

# 기본 통계 확인
print(f"\n기본 통계:")
print(f"- 분석 기간: {group_boards['created_at'].min().strftime('%Y-%m-%d')} ~ {group_boards['created_at'].max().strftime('%Y-%m-%d')}")
print(f"- 참가자 가입 기간: {participants['joined_at'].min().strftime('%Y-%m-%d')} ~ {participants['joined_at'].max().strftime('%Y-%m-%d')}")

# 상태 및 역할 값 확인
print(f"\n공구 상태 종류: {group_boards['status'].unique()}")
print(f"참가자 역할 종류: {participants['role'].unique()}")
print(f"지역 종류: {group_boards['location'].unique()}")


기본 통계:
- 분석 기간: 2024-11-28 ~ 2025-05-29
- 참가자 가입 기간: 2024-11-28 ~ 2025-05-29

공구 상태 종류: ['마감임박' '모집마감' '공구성공' '모집중']
참가자 역할 종류: ['참여자' '리더']
지역 종류: ['수원' '대구' '대전' '부산' '울산' '서울' '광주' '인천']


## 거래 상태 요약 분석


### 전체 공구 진행 현황 요약


In [5]:
# 기본 KPI 계산
total_groups = len(group_boards)
completed_groups = len(group_boards[group_boards['status'] == '공구성공'])
failed_groups = len(group_boards[group_boards['status'] == '모집마감'])
pending_groups = len(group_boards[group_boards['status'].isin(['모집중', '마감임박'])])

# 마감률 및 실패율 계산
completion_rate = (completed_groups / total_groups) * 100 if total_groups > 0 else 0
failure_rate = (failed_groups / total_groups) * 100 if total_groups > 0 else 0

print("핵심 KPI 지표:")
print(f"{'=' * 50}")
print(f"총 공구 수: {total_groups:,}개")
print(f"완료 공구 수: {completed_groups:,}개")
print(f"실패 공구 수: {failed_groups:,}개")
print(f"진행중 공구 수: {pending_groups:,}개")
print(f"마감률 (완료/전체): {completion_rate:.1f}%")
print(f"실패율: {failure_rate:.1f}%")

핵심 KPI 지표:
총 공구 수: 500개
완료 공구 수: 131개
실패 공구 수: 113개
진행중 공구 수: 256개
마감률 (완료/전체): 20.0%
실패율: 20.0%


### 상태별 상세 분석


In [6]:
status_summary = group_boards['status'].value_counts()
status_percentage = group_boards['status'].value_counts(normalize=True) * 100

print(f"\n상태별 상세 현황:")
for status, count in status_summary.items():
  percentage = status_percentage[status]
  print(f"- {status}: {count:,}개 ({percentage:.1f}%)")


상태별 상세 현황:
- 마감임박: 134개 (20.0%)
- 공구성공: 131개 (20.0%)
- 모집중: 122개 (20.0%)
- 모집마감: 113개 (20.0%)


### 지역별 성과 분석


In [7]:
# 지역별 공구 수
location_counts = group_boards['location'].value_counts()
print(f"\n지역별 공구 수:")
for location, count in location_counts.items():
  print(f"- {location}: {count:,}개")

# 지역별 성공률 계산
location_success_rate = group_boards.groupby('location')['status'].apply(
    lambda x: (x == '공구성공').sum() / len(x) * 100
).sort_values(ascending=False)

print(f"\n지역별 성공률 (높은 순):")
for location, rate in location_success_rate.items():
  print(f"- {location}: {rate:.1f}%")

# 지역별 실패율 계산 (우선 개선 대상)
location_failure_rate = group_boards.groupby('location')['status'].apply(
    lambda x: (x == '모집마감').sum() / len(x) * 100
).sort_values(ascending=False)

print(f"\n지역별 실패율 (높은 순) - 우선 개선 대상:")
for location, rate in location_failure_rate.head().items():
  print(f"- {location}: {rate:.1f}%")


지역별 공구 수:
- 대전: 71개
- 대구: 70개
- 인천: 67개
- 울산: 64개
- 부산: 62개
- 수원: 60개
- 광주: 55개
- 서울: 51개

지역별 성공률 (높은 순):
- 수원: 30.0%
- 부산: 30.0%
- 인천: 20.0%
- 서울: 20.0%
- 울산: 20.0%
- 대구: 20.0%
- 대전: 20.0%
- 광주: 20.0%

지역별 실패율 (높은 순) - 우선 개선 대상:
- 광주: 30.0%
- 부산: 20.0%
- 대전: 20.0%
- 대구: 20.0%
- 울산: 10.0%


### 거래 상태 시각화


In [8]:
# 공구 상태 분포 파이차트
fig_status = px.pie(
    values=status_summary.values,
    names=status_summary.index,
    title="전체 공구 상태 분포",
    color_discrete_sequence=px.colors.qualitative.Set3
)
fig_status.update_traces(textposition='inside', textinfo='percent+label')
fig_status.update_layout(title_font_size=16, width=600, height=500)
fig_status.show()

# 지역별 성공률 바차트
fig_location_success = px.bar(
    x=location_success_rate.index,
    y=location_success_rate.values,
    title="지역별 공구 성공률",
    labels={'x': '지역', 'y': '성공률 (%)'},
    color=location_success_rate.values,
    color_continuous_scale='Greens'
)
fig_location_success.update_layout(title_font_size=16, width=800, height=500)
fig_location_success.show()

# 지역별 실패율 바차트 (우선 개선 대상)
fig_location_failure = px.bar(
    x=location_failure_rate.index,
    y=location_failure_rate.values,
    title="지역별 공구 실패율 - 우선 개선 대상",
    labels={'x': '지역', 'y': '실패율 (%)'},
    color=location_failure_rate.values,
    color_continuous_scale='Reds'
)
fig_location_failure.update_layout(title_font_size=16, width=800, height=500)
fig_location_failure.show()

## 최근 거래 추이 분석


### 월별 거래 흐름 분석


In [9]:
# 월별 공구 개설 수
monthly_creation = group_boards.groupby('created_month').size()
print(f"월별 공구 개설 수:")
for month, count in monthly_creation.items():
  print(f"- {month}: {count:,}개")

# 월별 공구 완료 수
monthly_completion = group_boards[group_boards['status'] == '공구성공'].groupby('created_month').size()
print(f"\n월별 공구 완료 수:")
for month, count in monthly_completion.items():
  print(f"- {month}: {count:,}개")

# 월별 신규 리더 수
monthly_new_leaders = participants[participants['role'] == '리더'].groupby('joined_month').size()
print(f"\n월별 신규 리더 수:")
for month, count in monthly_new_leaders.items():
  print(f"- {month}: {count:,}명")

# 월별 전체 참가자 수
monthly_participants = participants.groupby('joined_month').size()
print(f"\n월별 전체 참가자 수:")
for month, count in monthly_participants.items():
  print(f"- {month}: {count:,}명")

월별 공구 개설 수:
- 2024-11: 12개
- 2024-12: 89개
- 2025-01: 64개
- 2025-02: 86개
- 2025-03: 78개
- 2025-04: 98개
- 2025-05: 73개

월별 공구 완료 수:
- 2024-11: 2개
- 2024-12: 28개
- 2025-01: 24개
- 2025-02: 25개
- 2025-03: 11개
- 2025-04: 21개
- 2025-05: 20개

월별 신규 리더 수:
- 2024-11: 15명
- 2024-12: 262명
- 2025-01: 259명
- 2025-02: 239명
- 2025-03: 247명
- 2025-04: 265명
- 2025-05: 220명

월별 전체 참가자 수:
- 2024-11: 34명
- 2024-12: 521명
- 2025-01: 503명
- 2025-02: 482명
- 2025-03: 493명
- 2025-04: 523명
- 2025-05: 444명


### 사용자당 평균 공구 참여 수


In [10]:
# 사용자별 참여 횟수 계산
user_participation = participants.groupby('user_id').size()
avg_participation_per_user = user_participation.mean()

print(f"\n참가자 참여 현황:")
print(f"- 사용자당 평균 공구 참여 수: {avg_participation_per_user:.2f}회")
print(f"- 최대 참여 횟수: {user_participation.max()}회")
print(f"- 1회만 참여한 사용자: {(user_participation == 1).sum():,}명")
print(f"- 다회 참여 사용자: {(user_participation > 1).sum():,}명")


참가자 참여 현황:
- 사용자당 평균 공구 참여 수: 10.00회
- 최대 참여 횟수: 22회
- 1회만 참여한 사용자: 0명
- 다회 참여 사용자: 300명


### 최근 3주간 공구 참여율 분석


In [11]:
# 최근 3주 기준일 계산
latest_date = participants['joined_at'].max()
three_weeks_ago = latest_date - timedelta(weeks=3)

print(f"\n최근 3주간 활동 분석 ({three_weeks_ago.strftime('%Y-%m-%d')} ~ {latest_date.strftime('%Y-%m-%d')}):")

# 최근 3주간 데이터 필터링
recent_participants = participants[participants['joined_at'] >= three_weeks_ago]
recent_groups = group_boards[group_boards['created_at'] >= three_weeks_ago]

print(f"- 최근 3주 신규 참가자: {len(recent_participants):,}명")
print(f"- 최근 3주 신규 공구: {len(recent_groups):,}개")
print(f"- 일평균 참가자: {len(recent_participants) / 21:.1f}명/일")
print(f"- 일평균 공구 생성: {len(recent_groups) / 21:.1f}개/일")


최근 3주간 활동 분석 (2025-05-08 ~ 2025-05-29):
- 최근 3주 신규 참가자: 347명
- 최근 3주 신규 공구: 58개
- 일평균 참가자: 10.0명/일
- 일평균 공구 생성: 2.0개/일


### 월별 거래 추이 시각화


In [12]:
# 월별 지표 종합 차트
fig_monthly = make_subplots(
    rows=2, cols=2,
    subplot_titles=['월별 공구 개설 수', '월별 공구 완료 수', '월별 신규 리더 수', '월별 참가자 수'],
    vertical_spacing=0.12
)

# 월별 공구 개설 수
fig_monthly.add_trace(
    go.Scatter(
        x=monthly_creation.index.astype(str),
        y=monthly_creation.values,
        mode='lines+markers',
        name="개설수",
        line=dict(color='blue', width=3)
    ),
    row=1, col=1
)

# 월별 공구 완료 수
fig_monthly.add_trace(
    go.Scatter(
        x=monthly_completion.index.astype(str),
        y=monthly_completion.values,
        mode='lines+markers',
        name="완료수",
        line=dict(color='green', width=3)
    ),
    row=1, col=2
)

# 월별 신규 리더 수
fig_monthly.add_trace(
    go.Bar(
        x=monthly_new_leaders.index.astype(str),
        y=monthly_new_leaders.values,
        name="신규리더",
        marker_color='orange'
    ),
    row=2, col=1
)

# 월별 참가자 수
fig_monthly.add_trace(
    go.Bar(
        x=monthly_participants.index.astype(str),
        y=monthly_participants.values,
        name="참가자수",
        marker_color='purple'
    ),
    row=2, col=2
)

fig_monthly.update_layout(
    title="월별 거래 흐름 분석",
    title_font_size=18,
    height=800,
    width=1200,
    showlegend=False
)
fig_monthly.show()

### 거래 증가 시기 파악


In [13]:
# 월별 총 활동량 (공구 수 + 참가자 수)
monthly_total_activity = monthly_creation + monthly_participants
monthly_growth_rate = monthly_total_activity.pct_change() * 100

print(f"\n거래 증가 시기 분석:")
print(f"월별 활동 증가율:")
for month, growth in monthly_growth_rate.dropna().items():
  trend = "급성장" if growth > 20 else "성장" if growth > 0 else "감소"
  print(f"- {month}: {growth:+.1f}% {trend}")

# 피크 시즌 식별
peak_months = monthly_total_activity.nlargest(3)
print(f"\n거래량 피크 시즌 (마케팅 집중 시기):")
for month, activity in peak_months.items():
  print(f"- {month}: 총 활동 {activity:,}건")


거래 증가 시기 분석:
월별 활동 증가율:
- 2024-12: +1000.0% 급성장
- 2025-01: -7.0% 감소
- 2025-02: +0.1% 성장
- 2025-03: +0.5% 성장
- 2025-04: +8.0% 성장
- 2025-05: -10.0% 감소

거래량 피크 시즌 (마케팅 집중 시기):
- 2025-04: 총 활동 621건
- 2024-12: 총 활동 610건
- 2025-03: 총 활동 571건


### 리더 유입 캠페인 필요성 분석


In [14]:
# 리더 비율 계산
total_leaders = len(participants[participants['role'] == '리더'])
total_participants_count = len(participants)
leader_ratio = (total_leaders / total_participants_count) * 100

print(f"\n리더 유입 현황 분석:")
print(f"- 전체 리더 수: {total_leaders:,}명")
print(f"- 전체 참가자 수: {total_participants_count:,}명")
print(f"- 리더 비율: {leader_ratio:.1f}%")

# 최근 리더 유입 추이
recent_leader_avg = monthly_new_leaders.tail(3).mean()
overall_leader_avg = monthly_new_leaders.mean()

campaign_needed = recent_leader_avg < overall_leader_avg * 0.8

print(f"\n리더 유입 캠페인 필요성:")
print(f"- 최근 3개월 평균 리더 유입: {recent_leader_avg:.1f}명/월")
print(f"- 전체 기간 평균 리더 유입: {overall_leader_avg:.1f}명/월")
print(f"- 캠페인 필요: {'예 (리더 유입 저조)' if campaign_needed else '아니오 (정상 수준)'}")


리더 유입 현황 분석:
- 전체 리더 수: 1,507명
- 전체 참가자 수: 3,000명
- 리더 비율: 50.0%

리더 유입 캠페인 필요성:
- 최근 3개월 평균 리더 유입: 200.0명/월
- 전체 기간 평균 리더 유입: 200.0명/월
- 캠페인 필요: 아니오 (정상 수준)


## 종합 KPI 대시보드


In [15]:
# 핵심 KPI 정리
kpi_summary = {
    '총 공구 수': f"{total_groups:,}개",
    '완료율': f"{completion_rate:.1f}%",
    '실패율': f"{failure_rate:.1f}%",
    '월평균 신규 공구': f"{monthly_creation.mean():.1f}개",
    '월평균 신규 리더': f"{monthly_new_leaders.mean():.1f}명",
    '리더 비율': f"{leader_ratio:.1f}%",
    '사용자당 평균 참여': f"{avg_participation_per_user:.2f}회",
    '최고 성과 지역': location_success_rate.index[0],
    '개선 필요 지역': location_failure_rate.index[0],
    '피크 시즌': str(peak_months.index[0])
}

print("핵심 성과 지표 (KPI):")
print("=" * 50)
for key, value in kpi_summary.items():
  print(f"{key}: {value}")

핵심 성과 지표 (KPI):
총 공구 수: 500개
완료율: 20.0%
실패율: 20.0%
월평균 신규 공구: 70.0개
월평균 신규 리더: 200.0명
리더 비율: 50.0%
사용자당 평균 참여: 10.00회
최고 성과 지역: 수원
개선 필요 지역: 광주
피크 시즌: 2025-04


### KPI 대시보드 시각화


In [16]:
# KPI 요약 테이블
fig_kpi = go.Figure(data=[go.Table(
    header=dict(
        values=['핵심 지표', '현재 값', '상태 평가'],
        fill_color='lightblue',
        font=dict(color='white', size=14),
        align='center'
    ),
    cells=dict(
        values=[
            ['총 공구 수', '완료율', '실패율', '월평균 신규 공구', '월평균 신규 리더', '리더 비율'],
            [f"{total_groups:,}개", f"{completion_rate:.1f}%", f"{failure_rate:.1f}%",
             f"{monthly_creation.mean():.1f}개", f"{monthly_new_leaders.mean():.1f}명", f"{leader_ratio:.1f}%"],
            ['정상', '모니터링 필요', '주의', '양호',
             '캠페인 필요' if campaign_needed else '양호', '개선 여지']
        ],
        fill_color='lightgrey',
        font=dict(size=12),
        align='center'
    )
)])

fig_kpi.update_layout(
    title="공구 플랫폼 핵심 지표 요약",
    title_font_size=20,
    width=900,
    height=400
)
fig_kpi.show()

## 운영 인사이트


### 데이터 기반 인사이트


In [17]:
print("주요 운영 인사이트:")
print("=" * 50)

insights = []

# 1. 지역별 성과 인사이트
best_region = location_success_rate.index[0]
worst_region = location_failure_rate.index[0]
insights.append(f"{best_region} 지역이 성공률 {location_success_rate.iloc[0]:.1f}%로 최고 성과")
insights.append(f"{worst_region} 지역이 실패율 {location_failure_rate.iloc[0]:.1f}%로 개선 시급")

# 2. 시즌별 인사이트
peak_season = str(peak_months.index[0])
insights.append(f"{peak_season}이 최대 활동 시즌 - 마케팅 집중 시기")

# 3. 리더 유입 인사이트
if campaign_needed:
  insights.append("리더 유입 저조로 리더 유치 캠페인 필요")
else:
  insights.append("리더 유입 안정적 - 현재 전략 유지")

# 4. 참여 패턴 인사이트
repeat_users_ratio = ((user_participation > 1).sum() / len(user_participation)) * 100
insights.append(f"재참여율 {repeat_users_ratio:.1f}% - 사용자 충성도 {'양호' if repeat_users_ratio > 30 else '개선 필요'}")

for i, insight in enumerate(insights, 1):
  print(f"{i}. {insight}")

주요 운영 인사이트:
1. 수원 지역이 성공률 30.0%로 최고 성과
2. 광주 지역이 실패율 30.0%로 개선 시급
3. 2025-04이 최대 활동 시즌 - 마케팅 집중 시기
4. 리더 유입 안정적 - 현재 전략 유지
5. 재참여율 100.0% - 사용자 충성도 양호


### 성과 개선 전략


In [18]:
print(f"\n성과 개선 전략:")
print("=" * 50)

print(f"""
완료율 향상 전략:
- 현재 완료율 {completion_rate:.1f}% → 목표 35% 이상
- {worst_region} 지역 집중 지원으로 전체 완료율 5%p 향상 가능
- 공구 진행 가이드 제공으로 성공률 증대

리더 확보 전략:
- 현재 리더 비율 {leader_ratio:.1f}% → 목표 30% 이상
- 리더 인센티브 프로그램으로 월 {monthly_new_leaders.mean():.0f}명 → {monthly_new_leaders.mean() * 1.5:.0f}명 목표
- 성공 리더 추천 시스템 도입

재참여율 향상 전략:
- 현재 재참여율 {repeat_users_ratio:.1f}% → 목표 40% 이상
- 맞춤형 공구 추천으로 사용자별 평균 참여 {avg_participation_per_user:.1f}회 → 3회 목표
- 로열티 프로그램 도입

지역 균형 발전 전략:
- {best_region} 지역 성공 모델을 {worst_region} 지역에 적용
- 지역별 특화 상품군 개발
- 지역 리더 양성 프로그램 운영
""")


성과 개선 전략:

완료율 향상 전략:
- 현재 완료율 20.0% → 목표 35% 이상
- 광주 지역 집중 지원으로 전체 완료율 5%p 향상 가능
- 공구 진행 가이드 제공으로 성공률 증대

리더 확보 전략:
- 현재 리더 비율 50.0% → 목표 30% 이상
- 리더 인센티브 프로그램으로 월 200명 → 300명 목표
- 성공 리더 추천 시스템 도입

재참여율 향상 전략:
- 현재 재참여율 100.0% → 목표 40% 이상
- 맞춤형 공구 추천으로 사용자별 평균 참여 10.0회 → 3회 목표
- 로열티 프로그램 도입

지역 균형 발전 전략:
- 수원 지역 성공 모델을 광주 지역에 적용
- 지역별 특화 상품군 개발
- 지역 리더 양성 프로그램 운영



### 모니터링 지표


In [19]:
print(f"\n지속 모니터링 지표:")
print("=" * 50)

monitoring_metrics = [
    "일일 신규 공구 생성 수",
    "주간 완료율 추이",
    "월간 리더 유입 수",
    "지역별 성공률 변화",
    "사용자 재참여율",
    "피크 시즌 대비 현재 활동 수준"
]

for i, metric in enumerate(monitoring_metrics, 1):
  print(f"{i}. {metric}")

print(f"""
경보 기준:
- 완료율 {completion_rate * .8:.1f}% 이하 시 알림
- 리더 유입 {monthly_new_leaders.mean() * .7:.0f}명/월 이하 시 캠페인 발동
- 특정 지역 실패율 40% 이상 시 긴급 대응
- 일일 신규 공구 {len(recent_groups) / 21 * .5:.1f}개 이하 시 마케팅 강화
""")


지속 모니터링 지표:
1. 일일 신규 공구 생성 수
2. 주간 완료율 추이
3. 월간 리더 유입 수
4. 지역별 성공률 변화
5. 사용자 재참여율
6. 피크 시즌 대비 현재 활동 수준

경보 기준:
- 완료율 20.0% 이하 시 알림
- 리더 유입 100명/월 이하 시 캠페인 발동
- 특정 지역 실패율 40% 이상 시 긴급 대응
- 일일 신규 공구 1.0개 이하 시 마케팅 강화



## 예측 및 목표 설정


In [20]:
# 성장 전망
current_monthly_avg = monthly_creation.mean()
target_growth_rate = 20  # 20% 성장 목표

print(f"3개월 후 예측 목표:")
print(f"- 현재 월평균 공구: {current_monthly_avg:.1f}개")
print(f"- 목표 월평균 공구: {current_monthly_avg * (1 + target_growth_rate / 100):.1f}개 (+{target_growth_rate}%)")
print(f"- 목표 완료율: {completion_rate + 5:.1f}% (+5%p)")
print(f"- 목표 리더 비율: {leader_ratio + 5:.1f}% (+5%p)")

3개월 후 예측 목표:
- 현재 월평균 공구: 70.0개
- 목표 월평균 공구: 80.0개 (+20%)
- 목표 완료율: 30.0% (+5%p)
- 목표 리더 비율: 50.0% (+5%p)
